<a href="https://colab.research.google.com/github/OmidGhadami95/Caregivers/blob/main/Caregivers_Narrative_features_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chi-square p-value 0.1 threshold

#Preprocessing

In [ ]:
!pip install --upgrade --force-reinstall numpy pandas scipy scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully

In [329]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import chi2
# !pip install --force-reinstall pandas scipy
# !python -m venv myenv
# !source myenv/bin/activate  # On Windows, use 'myenv\Scripts\activate'

!pip install gensim
from gensim.models import Word2Vec
print("NumPy version:", np.__version__)  # Should show 1.24.0 or higher
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stops = set(stopwords.words("english"))
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, f1_score

# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

NumPy version: 1.26.4


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [330]:
survey_df = pd.read_csv("/content/Depressedscore.csv")
survey_df.head()

,ID,PHQ Score,ZB Score,Depressed,Participant Text
0,1,8,28,0,"Yeah. Can I read the questions first? OK. Uh, ..."
1,2,15,17,1,"OK. OK, so my name is Jonathan. I'm Fergal. 25..."
2,3,2,24,0,"Alright, my full name is Brian Brian Hemsworth..."
3,4,11,26,1,"Yeah, my name is Montana Franklin. I'm 26 year..."
4,5,21,29,1,OK. I'm Victor Daniel. My name is Victor Danie...


In [331]:
corrected_df=pd.read_csv("/content/correct_text.csv")
corrected_df.head()

,Unnamed: 0,Actual_text,Corrected_text
0,0,"[""Yeah. Can I read the questions first? OK. Uh...","['\n [/INST] ""Yeah. Can I read the questions ..."
1,1,"[""OK. OK, so my name is Jonathan. I'm Fergal. ...","[""\n [/INST] I'm Jonathan. I'm Fergal, and I'..."
2,2,"[""Alright, my full name is Brian Brian Hemswor...","[""\n [/INST] Alright, my full name is Brian H..."
3,3,"[""Yeah, my name is Montana Franklin. I'm 26 ye...","['\n [/INST] Yeah, my name is Montana Frankli..."
4,4,"[""OK. I'm Victor Daniel. My name is Victor Dan...","[""\n [/INST] I'm Victor Daniel. I'm 27 years ..."


In [332]:
survey_df["corrected_text"] = corrected_df["Corrected_text"]
survey_df.head()

,ID,PHQ Score,ZB Score,Depressed,Participant Text,corrected_text
0,1,8,28,0,"Yeah. Can I read the questions first? OK. Uh, ...","['\n [/INST] ""Yeah. Can I read the questions ..."
1,2,15,17,1,"OK. OK, so my name is Jonathan. I'm Fergal. 25...","[""\n [/INST] I'm Jonathan. I'm Fergal, and I'..."
2,3,2,24,0,"Alright, my full name is Brian Brian Hemsworth...","[""\n [/INST] Alright, my full name is Brian H..."
3,4,11,26,1,"Yeah, my name is Montana Franklin. I'm 26 year...","['\n [/INST] Yeah, my name is Montana Frankli..."
4,5,21,29,1,OK. I'm Victor Daniel. My name is Victor Danie...,"[""\n [/INST] I'm Victor Daniel. I'm 27 years ..."


In [333]:
survey_df['High Burden'] = (survey_df['ZB Score'] >= 20).astype(int)
# Get the name of the last column
last_column = survey_df.columns[-1]

# Get all column names except the last one
other_columns = survey_df.columns[:-1].tolist()

# Create a new column order
new_order = other_columns[:4] + [last_column] + other_columns[4:]

# Reorder the DataFrame
survey_df = survey_df[new_order]

survey_df.head()

,ID,PHQ Score,ZB Score,Depressed,High Burden,Participant Text,corrected_text
0,1,8,28,0,1,"Yeah. Can I read the questions first? OK. Uh, ...","['\n [/INST] ""Yeah. Can I read the questions ..."
1,2,15,17,1,0,"OK. OK, so my name is Jonathan. I'm Fergal. 25...","[""\n [/INST] I'm Jonathan. I'm Fergal, and I'..."
2,3,2,24,0,1,"Alright, my full name is Brian Brian Hemsworth...","[""\n [/INST] Alright, my full name is Brian H..."
3,4,11,26,1,1,"Yeah, my name is Montana Franklin. I'm 26 year...","['\n [/INST] Yeah, my name is Montana Frankli..."
4,5,21,29,1,1,OK. I'm Victor Daniel. My name is Victor Danie...,"[""\n [/INST] I'm Victor Daniel. I'm 27 years ..."


Stop word removal

In [334]:
import pandas as pd
import re

# Define your custom stop word list
custom_stopwords = {'the', 'a', 'an', 'and', 'or'}

# Define the stop word removal function
def remove_custom_stopwords(text):
    if pd.isna(text):
        return ""
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    words = text.split()
    filtered_words = [word for word in words if word not in custom_stopwords]
    return ' '.join(filtered_words)

# Apply it to your DataFrame
survey_df['Participant Text'] = survey_df['Participant Text'].apply(remove_custom_stopwords)

survey_df.head()

,ID,PHQ Score,ZB Score,Depressed,High Burden,Participant Text,corrected_text
0,1,8,28,0,1,yeah can i read questions first ok uh my name ...,"['\n [/INST] ""Yeah. Can I read the questions ..."
1,2,15,17,1,0,ok ok so my name is jonathan im fergal years o...,"[""\n [/INST] I'm Jonathan. I'm Fergal, and I'..."
2,3,2,24,0,1,alright my full name is brian brian hemsworth ...,"[""\n [/INST] Alright, my full name is Brian H..."
3,4,11,26,1,1,yeah my name is montana franklin im years old ...,"['\n [/INST] Yeah, my name is Montana Frankli..."
4,5,21,29,1,1,ok im victor daniel my name is victor daniel y...,"[""\n [/INST] I'm Victor Daniel. I'm 27 years ..."


Extract adjectives

In [335]:
import spacy
import re

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Process 'Participant Text' to find sentences with adjectives
sentences_with_adj = []
for text in survey_df['Participant Text']:
    doc = nlp(text)
    sentences = []
    for sent in doc.sents:
        if any(token.pos_ == 'ADJ' for token in sent):
            sentences.append(sent.text)

    sent_str = ''
    for i in sentences:
        i = i.lower()
        i = re.sub("[^a-zA-Z\_(SM\d)]", " ", i)
        sent_str += i + " "

    sentences_with_adj.append(sent_str.strip())

# Add the new column to the DataFrame
survey_df['ACD'] = sentences_with_adj

# Process 'Participant Text' to extract adjective phrases
adj_phrases = []
for text in survey_df['Participant Text']:
    doc = nlp(text)
    phrases = []
    for token in doc:
        if token.pos_ == 'ADJ':
            phrases.append(token.text.lower())
    adj_phrases.append(' '.join(phrases))

# Add the new column to the DataFrame
survey_df['ADJ'] = adj_phrases
survey_df.head()

,ID,PHQ Score,ZB Score,Depressed,High Burden,Participant Text,corrected_text,ACD,ADJ
0,1,8,28,0,1,yeah can i read questions first ok uh my name ...,"['\n [/INST] ""Yeah. Can I read the questions ...",yeah can i read questions first ok uh my name ...,palma real few artificial helpful loved many a...
1,2,15,17,1,0,ok ok so my name is jonathan im fergal years o...,"[""\n [/INST] I'm Jonathan. I'm Fergal, and I'...",ok ok so my name is jonathan im fergal years o...,fergal old biological old close same primary s...
2,3,2,24,0,1,alright my full name is brian brian hemsworth ...,"[""\n [/INST] Alright, my full name is Brian H...",alright my full name is brian brian hemsworth ...,full old easy more much ill best problematic f...
3,4,11,26,1,1,yeah my name is montana franklin im years old ...,"['\n [/INST] Yeah, my name is Montana Frankli...",yeah my name is montana franklin im years old ...,old dead easy ready same same same clinical se...
4,5,21,29,1,1,ok im victor daniel my name is victor daniel y...,"[""\n [/INST] I'm Victor Daniel. I'm 27 years ...",ok im victor daniel my name is victor daniel y...,old challenging past ok social religious posit...


**I preprocessed participant text text instead of participant text to reach the preprocess_text_orginal_wo_lemma. Also, Mahesh preprocessed participant text to reach the preprocess_text_original_wo_lemma**

Removes various newline characters and special characters

Converts text to lowercase

Removes extra whitespace

Uses spaCy for tokenization

In [336]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess text for each row in the DataFrame
preprocessed_texts = []
for text in survey_df['Participant Text']:
    # Text cleaning
    text = text.replace(" \n ", "").replace("'\n ", "").replace('"\n', "")
    text = text.replace("[/INST] ", "").replace("[/INST] ", "")
    text = text.replace("[", "").replace("]", "")
    text = text.replace("\\n", " ")
    text = re.sub("[^a-zA-Z\_(SM\d)]", " ", text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)

    # Process the text with spaCy
    doc = nlp(text)

    # Extract words, removing only the specific stopwords
    words = [token.text for token in doc]

    # Join the words back into a single string
    preprocessed_text = " ".join(words)

    preprocessed_texts.append(preprocessed_text)

# Add the new column to the DataFrame
survey_df['UNI'] = preprocessed_texts

In [337]:
survey_df.head()

,ID,PHQ Score,ZB Score,Depressed,High Burden,Participant Text,corrected_text,ACD,ADJ,UNI
0,1,8,28,0,1,yeah can i read questions first ok uh my name ...,"['\n [/INST] ""Yeah. Can I read the questions ...",yeah can i read questions first ok uh my name ...,palma real few artificial helpful loved many a...,yeah can i read questions first ok uh my name ...
1,2,15,17,1,0,ok ok so my name is jonathan im fergal years o...,"[""\n [/INST] I'm Jonathan. I'm Fergal, and I'...",ok ok so my name is jonathan im fergal years o...,fergal old biological old close same primary s...,ok ok so my name is jonathan i m fergal years ...
2,3,2,24,0,1,alright my full name is brian brian hemsworth ...,"[""\n [/INST] Alright, my full name is Brian H...",alright my full name is brian brian hemsworth ...,full old easy more much ill best problematic f...,alright my full name is brian brian hemsworth ...
3,4,11,26,1,1,yeah my name is montana franklin im years old ...,"['\n [/INST] Yeah, my name is Montana Frankli...",yeah my name is montana franklin im years old ...,old dead easy ready same same same clinical se...,yeah my name is montana franklin i m years old...
4,5,21,29,1,1,ok im victor daniel my name is victor daniel y...,"[""\n [/INST] I'm Victor Daniel. I'm 27 years ...",ok im victor daniel my name is victor daniel y...,old challenging past ok social religious posit...,ok i m victor daniel my name is victor daniel ...


In [338]:
survey_df.to_csv("preprocessed_narrative.csv", index=False)

Feature Selection

In [339]:
def feature_selection(data, target_column, drop_columns):
    # Create a copy of the data to avoid modifying the original
    df = data.copy()

    # Encode categorical variables using LabelEncoder
    label_encoders = {}
    for column in df.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column].astype(str))
        label_encoders[column] = le

    # Handle missing values by filling them with the mode of each column
    df.fillna(df.mode().iloc[0], inplace=True)

    # Separate features and target variable
    X = df.drop(columns=drop_columns + [target_column])
    y = df[target_column]

    # Perform Chi-square test
    chi2_scores, p_values = chi2(X, y)

    # Create results DataFrame
    chi2_results = pd.DataFrame({
        'Feature': X.columns,
        'Chi2 Score': chi2_scores,
        'P-Value': p_values
    })

    # Sort by p-value and filter significant features
    significant_features = chi2_results[chi2_results['P-Value'] < 0.1]\
                          .sort_values(by='P-Value', ascending=True)

    # Return all significant features
    return significant_features['Feature'].tolist(), chi2_results

# ADJ

##LR

High Burden

In [340]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ADJ"]
    labels = data["High Burden"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize text within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        model = LogisticRegression(max_iter=1000)
        model.fit(X_train, y_train)
        pred = model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



Depressed

In [341]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ADJ"]
    labels = data["Depressed"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize text within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        model = LogisticRegression(max_iter=1000)
        model.fit(X_train, y_train)
        pred = model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



## SVM

High Burden

In [342]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ADJ"]
    labels = data["High Burden"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize text within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        model = SVC(kernel='linear')
        model.fit(X_train, y_train)
        pred = model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



Depressed

In [343]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ADJ"]
    labels = data["Depressed"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize text within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        model = SVC(kernel='linear')
        model.fit(X_train, y_train)
        pred = model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



## KNN

Burden

In [344]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ADJ"]
    labels = data["High Burden"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize text within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        model = KNeighborsClassifier(n_neighbors=5)
        model.fit(X_train, y_train)
        pred = model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



Depressed

In [345]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ADJ"]
    labels = data["Depressed"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize text within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        model = KNeighborsClassifier(n_neighbors=5)
        model.fit(X_train, y_train)
        pred = model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()


--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



##Random Forest

High Burden

In [346]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ADJ"]
    labels = data["High Burden"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize text within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        model = RandomForestClassifier(n_estimators=100, random_state=32)
        model.fit(X_train, y_train)
        pred = model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



Depressed

In [347]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ADJ"]
    labels = data["Depressed"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize text within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        pred = model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()


--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



# UNI

##LR

High Burden

In [348]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["UNI"]
    labels = data["High Burden"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize within fold
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        logistic_model = LogisticRegression()
        logistic_model.fit(X_train, y_train)
        pred = logistic_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



--- Processing iteration 28/65 ---

--- Processing iteration 29/65 ---

--- Processing iteration 30/65 ---

--- Processing iteration 31/65 ---

--- Processing iteration 32/65 ---

--- Processing iteration 33/65 ---

--- Processing iteration 34/65 ---

--- Processing iteration 35/65 ---

--- Processing iteration 36/65 ---

--- Processing iteration 37/65 ---

--- Processing iteration 38/65 ---

--- Processing iteration 39/65 ---

--- Processing iteration 40/65 ---

--- Processing iteration 41/65 ---

--- Processing iteration 42/65 ---

--- Processing iteration 43/65 ---

--- Processing iteration 44/65 ---

--- Processing iteration 45/65 ---

--- Processing iteration 46/65 ---

--- Processing iteration 47/65 ---

--- Processing iteration 48/65 ---

--- Processing iteration 49/65 ---

--- Processing iteration 50/65 ---

--- Processing iteration 51/65 ---

--- Processing iteration 52/65 ---

--- Processing iteration 53/65 ---

--- Processing iteration 54/65 ---

--- Processing iteration 55

Depressed

In [349]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["UNI"]
    labels = data["Depressed"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize within fold
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        logistic_model = LogisticRegression()
        logistic_model.fit(X_train, y_train)
        pred = logistic_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



## SVM

High Burden

In [350]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["UNI"]
    labels = data["High Burden"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        svm_model = SVC(kernel='linear')
        svm_model.fit(X_train, y_train)
        pred = svm_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



Depressed

In [352]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["UNI"]
    labels = data["Depressed"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        svm_model = SVC(kernel='linear')
        svm_model.fit(X_train, y_train)
        pred = svm_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



## KNN

Burden

In [316]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["UNI"]
    labels = data["High Burden"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        knn_model = KNeighborsClassifier(n_neighbors=5)
        knn_model.fit(X_train, y_train)
        pred = knn_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



Depressed

In [317]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["UNI"]
    labels = data["Depressed"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        knn_model = KNeighborsClassifier(n_neighbors=5)
        knn_model.fit(X_train, y_train)
        pred = knn_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



## Random Forest

High Burden

In [318]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline using Random Forest and no feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["UNI"]
    labels = data["High Burden"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_model.fit(X_train, y_train)
        pred = rf_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



Depressed

In [319]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline using Random Forest and no feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["UNI"]
    labels = data["Depressed"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_model.fit(X_train, y_train)
        pred = rf_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



#ACD

##LR

High Burden

In [320]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ACD"]
    labels = data["High Burden"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize within fold
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        logistic_model = LogisticRegression()
        logistic_model.fit(X_train, y_train)
        pred = logistic_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



Depressed

In [321]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
import pandas as pd

# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ACD"]
    labels = data["Depressed"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize within fold
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        logistic_model = LogisticRegression()
        logistic_model.fit(X_train, y_train)
        pred = logistic_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



## SVM

High Burden

In [322]:
# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ACD"]
    labels = data["High Burden"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        svm_model = SVC(kernel='linear')
        svm_model.fit(X_train, y_train)
        pred = svm_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



Depressed

In [323]:
# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ACD"]
    labels = data["Depressed"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        svm_model = SVC(kernel='linear')
        svm_model.fit(X_train, y_train)
        pred = svm_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



## KNN

Burden

In [324]:
# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ACD"]
    labels = data["High Burden"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize text within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        knn_model = KNeighborsClassifier(n_neighbors=5)
        knn_model.fit(X_train, y_train)
        pred = knn_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the
run_pipeline()


--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



Depressed

In [325]:
# Modified pipeline without feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ACD"]
    labels = data["Depressed"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize text within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        knn_model = KNeighborsClassifier(n_neighbors=5)
        knn_model.fit(X_train, y_train)
        pred = knn_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



##Random Forest

High Burden

In [326]:
# Modified pipeline using Random Forest and no feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ACD"]
    labels = data["High Burden"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize text within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_model.fit(X_train, y_train)
        pred = rf_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---



Depressed

In [327]:
# Modified pipeline using Random Forest and no feature selection
def run_pipeline():
    # Load data
    data = pd.read_csv("preprocessed_narrative.csv")
    texts = data["ACD"]
    labels = data["Depressed"]

    # Initialize CV and model
    loo = LeaveOneOut()
    predictions = []
    true_labels = []

    for iteration, (train_idx, test_idx) in enumerate(loo.split(texts)):
        print(f"\n--- Processing iteration {iteration+1}/{len(texts)} ---")

        # Split raw text data
        X_train_raw, X_test_raw = texts.iloc[train_idx], texts.iloc[test_idx]
        y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

        # Vectorize text within fold (BoW)
        vectorizer = CountVectorizer()
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        # Train and predict using all features
        rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_model.fit(X_train, y_train)
        pred = rf_model.predict(X_test)

        predictions.append(pred[0])
        true_labels.append(y_test.values[0])

    # Calculate final metrics
    f1 = f1_score(true_labels, predictions, average='binary')
    precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
    recall = recall_score(true_labels, predictions, average='binary', zero_division=0)

    print(f"\nFinal LOOCV F1 Score: {f1:.2f}")
    print(f"Final LOOCV Precision: {precision:.2f}")
    print(f"Final LOOCV Recall: {recall:.2f}")

# Run the pipeline
run_pipeline()



--- Processing iteration 1/65 ---

--- Processing iteration 2/65 ---

--- Processing iteration 3/65 ---

--- Processing iteration 4/65 ---

--- Processing iteration 5/65 ---

--- Processing iteration 6/65 ---

--- Processing iteration 7/65 ---

--- Processing iteration 8/65 ---

--- Processing iteration 9/65 ---

--- Processing iteration 10/65 ---

--- Processing iteration 11/65 ---

--- Processing iteration 12/65 ---

--- Processing iteration 13/65 ---

--- Processing iteration 14/65 ---

--- Processing iteration 15/65 ---

--- Processing iteration 16/65 ---

--- Processing iteration 17/65 ---

--- Processing iteration 18/65 ---

--- Processing iteration 19/65 ---

--- Processing iteration 20/65 ---

--- Processing iteration 21/65 ---

--- Processing iteration 22/65 ---

--- Processing iteration 23/65 ---

--- Processing iteration 24/65 ---

--- Processing iteration 25/65 ---

--- Processing iteration 26/65 ---

--- Processing iteration 27/65 ---

--- Processing iteration 28/65 ---

